# Persona Retrieval Experiment



## Local setup

1. Activate your virtual environment and install dependencies with `make install`.
2. Run the setup cells below to make sure the notebook can locate `src/` when opened outside the project root.


Load Jupyter extensions that keep modules fresh while you iterate.


In [1]:
%load_ext autoreload
%autoreload 2


Resolve the repository root so imports and data paths work whether you open the notebook from `notebooks/` or elsewhere.


In [2]:
import os
import sys
from pathlib import Path

REPO_ROOT = Path(os.getenv("ADSP_REPO_ROOT", Path.cwd())).resolve().parent


Load the shared persona description and the indicators used to drive retrieval.


In [3]:
from adsp.data_pipeline.schema import load_persona_profile

persona = load_persona_profile(REPO_ROOT / "data/processed/personas/common_traits/curious-connoisseurs.json")
persona_indicator = load_persona_profile(REPO_ROOT / "data/processed/personas/individual/curious-connoisseurs.json")


Load environment variables from the project `.env` file.


In [4]:
from dotenv import load_dotenv

load_dotenv(REPO_ROOT / ".env")


True

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

## load the embedding model
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)


/home/thanhnv/polito_local/source/2025/adsp/2025-P3-AI-Personas/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
## Instantiate the RAG helper that wraps persona indexing and search utilities.

from langchain_core.vectorstores import InMemoryVectorStore
from adsp.data_pipeline.persona_data_pipeline import PersonaIndicatorRAG, documents_to_context_prompt

vector_store = InMemoryVectorStore(embedding=embedding_model)
persona_rag = PersonaIndicatorRAG(embedding_model, vectorstore=vector_store)


In [7]:
## Index the persona indicators so they are ready for semantic search.

persona_rag.index_persona(persona_indicator)


['dee59f80-83c4-4d96-bacf-374ce7b10893',
 'bdd18b75-5218-4f8c-ba96-611791dfcceb',
 '13382c51-e998-439e-9921-55591a69bb2c',
 '7ace46b5-6629-4ca4-bd4c-4d423095e45d',
 'b9f0f9b1-e744-4be6-a6df-008aedd3e725',
 '84937f47-d849-42f7-bcdc-63de037ebc1d',
 'c69350d1-b455-40aa-97b3-d9a47f44a92b',
 '98cfe8da-2f7b-4a26-aa54-420a592024c5',
 '11cb6c6f-362d-42ac-a6a0-c61015a85d59',
 '94d1102d-0e25-4c53-a706-beb70737676e',
 '7710eb9b-93cf-4983-8ff1-cca85998b003',
 '631f83c2-2a69-47c9-91f3-0df8927cfc66',
 'cfc986ea-8fef-4131-8481-48ca2dae60aa',
 'b7d1d40e-6312-4776-beab-cdfb755f5ae4',
 '5485e180-b82a-4211-82c1-078716e72682',
 '2ce30e26-a8ea-432f-8499-c6e54df5b579',
 '7c87c7a2-77bb-4eee-879f-ed299a51f5be',
 '1ee79cd0-cda0-4d76-b633-52f3825c93e3',
 '3599e90a-2b16-474b-82e9-b3c3201a79e5',
 '7c9620b9-57ca-4637-8911-f08d800c6661',
 'e26c9bf6-bd59-46a8-9a19-1bfd1a125649',
 'c253f47a-a375-4422-a4a5-c984d559355b',
 'ca7e38b6-bb95-4bae-acc5-5dc9661c3052',
 'bd529ffc-4327-4b09-a4d0-86b2e1cd3709',
 '496c593c-ba73-

In [8]:
from openai import OpenAI

# Connect to an LLM endpoint using NVIDIA's hosted API
api_key = os.getenv("NVIDIA_API_KEY")
base_url = "https://integrate.api.nvidia.com/v1"

client = OpenAI(base_url=base_url, api_key=api_key)


In [9]:
from adsp.core.prompt_builder import system_prompt

# Convert a persona profile (with reasoning traits) into a system prompt.
SYSTEM_PROMPT = system_prompt.persona_to_system_prompt(persona)
SYSTEM_PROMPT


'You are persona "Curious Connoisseurs (id: curious-connoisseurs)".\nSummary: Skew toward older groups, urban areas, high income, exploratory, knowledgeable, quality seeker, beans seeker, innovative.\n\nVoice:\n- Tone adjectives: knowledgeable, exploratory, innovative, enthusiastic, tech-savvy\n- Delivery: formality=medium; directness=balanced; emotional_flavour=enthusiastic; criticality=medium\n- Verbosity: detailed\n- Preferred structures: detailed descriptions, comparative analysis, exploratory narratives, technical specifications\n- Register examples: I like to try new coffee brands, I like to experiment with new or lesser known coffee brands, I like to discover new coffee types, I enjoy exploring the latest technology trends\n\nValue frame:\n- Priority order: quality > innovation > sustainability > convenience > price > technology integration\n- Sensitivities: sustainability=high; price_sensitivity=medium; novelty_seeking=high; brand_loyalty=medium; health_concern=medium\n- Summar

In [10]:
USER_QUERY = "In general, what is your age?"


In [11]:
## 1. Combine persona name and query into the retrieval input used by the RAG helper.
retrieval_query = f"persona: {persona.persona_name} | query: {USER_QUERY}"
## 2. Search the indexed indicators for the most relevant snippets.
retrieved_indicators = persona_rag.search(retrieval_query)
## 3. Convert retrieved documents into a context block.

context = documents_to_context_prompt(retrieved_indicators)

In [12]:
# Send the chat completion request with persona context and show the model response.
model = "mistralai/mistral-small-24b-instruct"
temperature = 0.5
top_p = 0.5
max_tokens = 4024
response_timeout = 300

system_prompt_with_context = f"{SYSTEM_PROMPT}\n\nPROVIDE CONTEXT: \n{context}"

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt_with_context},
        {"role": "user", "content": USER_QUERY},
    ],
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_tokens,
    timeout=response_timeout,
)
raw_text = response.choices[0].message.content or ""
raw_text


'As Curious Connoisseurs, we tend to skew toward older groups, with a significant portion of our demographic falling between 35 and 70 years old. Specifically, 35% are between 35-44 years, 15% are between 45-54 years, and 24% are between 55-70 years. Our average age is 47 years, reflecting a mature and experienced cohort. This age distribution aligns with our exploratory nature and appreciation for high-quality, innovative products.'